In [10]:
import torch
import random
import numpy as np
import pickle
import math

from torch import nn, optim, autograd

In [ ]:
INPUT_LAYER_DIM = 36
INPUT_LAYER_OUT = 256
SECOND_LAYER_OUT = 512
THIRD_LAYER_OUT = 1024
FOURTH_LAYER_OUT = 512
FIFTH_LAYER_OUT = 256
OUTPUT_LAYER_OUT = 36
T_MIN, T_MAX = 0, 36
BATCH_SIZE = 128
R_MIN, R_MAX = 0, 1
NUM_OF_SAMPLES = 2_000_000

use_cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(f"Device is {DEVICE}")

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

Device is cuda:0


In [3]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(INPUT_LAYER_DIM, INPUT_LAYER_OUT),
            nn.LeakyReLU(True),
            nn.Linear(INPUT_LAYER_OUT, SECOND_LAYER_OUT),
            nn.LeakyReLU(True),
            nn.Linear(SECOND_LAYER_OUT, THIRD_LAYER_OUT),
            nn.LeakyReLU(True),
            nn.Linear(THIRD_LAYER_OUT, FOURTH_LAYER_OUT),
            nn.LeakyReLU(True),
            nn.Linear(FOURTH_LAYER_OUT, FIFTH_LAYER_OUT),
            nn.LeakyReLU(True),
            nn.Linear(FIFTH_LAYER_OUT, OUTPUT_LAYER_OUT),
            nn.LeakyReLU()
        )

    def forward(self, x):
        output = self.net(x)
        return output.reshape(-1)

In [5]:
# Initialize model, optimizer, and loss_function (with dummy instances)
model = Generator().to(DEVICE)  # Replace YourModel() with your actual model architecture
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.0, 0.9), weight_decay=2e-5)

# Load saved model, optimizer, and loss
checkpoint = torch.load('/root/odev/generator_csprn/25-12-2023-210110/generator.pth')

# Load model and optimizer state dictionaries
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Ensure the model is in evaluation mode after loading
model.eval()

# Now you can use the loaded model, optimizer, and loss function for inference or further training


Generator(
  (net): Sequential(
    (0): Linear(in_features=36, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=True)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=True)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=True)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): LeakyReLU(negative_slope=True)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): LeakyReLU(negative_slope=True)
    (10): Linear(in_features=256, out_features=36, bias=True)
    (11): LeakyReLU(negative_slope=0.01)
  )
)

In [ ]:
# Non-random number dataset.
with open("/root/odev/data/tensor_data_list.pkl", "rb") as f:
    random_number_data_set = pickle.load(f)


In [12]:
# Generate and stack random numbers.
# In training float32 selected so 1 bit sign, 8 bits exponent and 23 bits mantissa.
NUM_TEST_BITS = 1_048_576
LEN_OF_ONE_OUTPUT_SEQ = 36
LEN_OF_MANTISSA = 23
NUM_ITER = math.ceil(NUM_TEST_BITS / (LEN_OF_ONE_OUTPUT_SEQ * LEN_OF_MANTISSA))
NUM_ITER

1267

In [19]:
res_num_list = []
for idx in range(NUM_ITER):
    # Make predictions
    input_data = random_number_data_set[idx]
    with torch.no_grad():
        predictions = model(input_data.to(torch.float32).to(DEVICE))
    res_num_list.append(predictions)

In [22]:
res_num_list[100]

tensor([0.8779, 0.9136, 0.8565, 0.7656, 0.7635, 0.9012, 0.8999, 0.7485, 0.8869,
        0.7685, 0.7808, 0.8041, 0.9401, 0.9580, 0.7869, 0.9172, 0.8030, 0.9376,
        0.8196, 0.8902, 0.8951, 0.8567, 0.6800, 0.8556, 0.8938, 0.6417, 0.7118,
        0.8322, 0.8010, 0.4426, 0.9400, 0.8528, 0.8888, 0.7972, 0.6270, 0.8427],
       device='cuda:0')

In [26]:
res_num_list[0].cpu().numpy()[0]

-0.01242662

In [32]:
import struct

# Example signed floating-point number
float_number = -123.456789

# Get the binary representation of the floating-point number
float_bytes = struct.pack('>f', float_number)

# Extract the mantissa bytes
mantissa_bytes = float_bytes[1:]  # Exclude the sign byte

# Convert mantissa bytes to binary string
mantissa_binary = ''.join(f"{byte:08b}" for byte in mantissa_bytes)

print("Mantissa Binary String:", mantissa_binary)


Mantissa Binary String: 111101101110100111100000


In [34]:
float_bytes[1:]

b'\xf6\xe9\xe0'

In [37]:
def generate_bin_str(data):
    
    res_str = ""
    for datum in data:
        for num in datum.cpu().numpy():
            float_bytes = struct.pack('>f', num)

            # Extract the components
            sign_bit = float_bytes[0] >> 7
            exponent = ((float_bytes[0] & 0x7F) << 1) | (float_bytes[1] >> 7)
            mantissa = ((float_bytes[1] & 0x7F) << 16) | (float_bytes[2] << 8) | float_bytes[3]

            # Convert components to binary strings
            sign_binary = bin(sign_bit)[2:]
            exponent_binary = bin(exponent)[2:].zfill(8)  # Ensure the exponent is 8 bits
            mantissa_binary = bin(mantissa)[2:].zfill(23)  # Ensure the mantissa is 23 bits

            res_str += mantissa_binary
    
    return res_str

res_bin_str = generate_bin_str(res_num_list)


In [43]:
with open("/root/odev/generated_random_number_binary.txt", "w") as f:
    f.write(res_bin_str)

In [45]:
import time
start = time.time()
with torch.no_grad():
    predictions = model.to("cpu")(input_data.to(torch.float32).to("cpu"))
end = time.time()
print(f"Elapsed time: {end - start}")

Elapsed time: 0.0020017623901367188
